In [0]:
%pylab inline

Загрузите выборку из файла gbm-data.csv с помощью pandas и преобразуйте ее в массив numpy (параметр values у датафрейма). В первой колонке файла с данными записано, была или нет реакция. Все остальные колонки (d1 - d1776) содержат различные характеристики молекулы, такие как размер, форма и т.д. Разбейте выборку на обучающую и тестовую, используя функцию train_test_split с параметрами test_size = 0.8 и random_state = 241.

In [0]:
import pandas as pd
from sklearn.cross_validation import train_test_split
data = pd.read_csv(open('gbm-data.csv'))
print(data.head())
X = data.values[:,1:]
y = data.values[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=241)
#print(X_train)
#print(y_train)
#print(X_test)
#print(y_test)

Обучите GradientBoostingClassifier с параметрами n_estimators=250, verbose=True, random_state=241 и для каждого значения learning_rate из списка [1, 0.5, 0.3, 0.2, 0.1] проделайте следующее:
- Используйте метод staged_decision_function для предсказания качества на обучающей и тестовой выборке на каждой итерации.
- Преобразуйте полученное предсказание с помощью сигмоидной функции по формуле 1 / (1 + e^{−y_pred}), где y_pred — предсказаное значение.
- Вычислите и постройте график значений log-loss (которую можно посчитать с помощью функции sklearn.metrics.log_loss) на обучающей и тестовой выборках, а также найдите минимальное значение метрики и номер итерации, на которой оно достигается.

Как можно охарактеризовать график качества на тестовой выборке, начиная с некоторой итерации: переобучение (overfitting) или недообучение (underfitting)? В ответе укажите одно из слов overfitting либо underfitting.

Приведите минимальное значение log-loss и номер итерации, на котором оно достигается, при learning_rate = 0.2.

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss
import math
import matplotlib.pylab as plt

with open("grad_boost.1.txt", "w") as file:
    file.write("overfitting")

for rate in (1.0, 0.5, 0.3, 0.2, 0.1):
    clf = GradientBoostingClassifier(learning_rate=rate, n_estimators=250, verbose=True, random_state=241)
    clf.fit(X_train, y_train)
    
    log_loss_test = np.empty(len(clf.estimators_))
    log_loss_train = np.empty(len(clf.estimators_))
    
    for i, pred in enumerate(clf.staged_decision_function(X_test)):
        log_loss_test[i] = log_loss(y_test, 1.0 / (1.0 + np.exp(-pred)))
        if rate == 0.2:
            print("#%d %f" % (i,log_loss_test[i]))
    for i, pred in enumerate(clf.staged_decision_function(X_train)):
        log_loss_train[i] = log_loss(y_train, 1.0 / (1.0 + np.exp(-pred)))
      
    if rate == 0.2:
        index = np.argsort(log_loss_test)[0]
        print("Best log-loss index=%d value=%.2f" % (index,log_loss_test[index]))
        with open("grad_boost.2.txt", "w") as file:
            file.write("%.2f %d" % (log_loss_test[index],index))
    
    plt.plot(log_loss_test)
    plt.plot(log_loss_train)
    plt.show()

На этих же данных обучите RandomForestClassifier с количеством деревьев, равным количеству итераций, на котором достигается наилучшее качество у градиентного бустинга из предыдущего пункта, c random_state=241 и остальными параметрами по умолчанию. Какое значение log-loss на тесте получается у этого случайного леса? (Не забывайте, что предсказания нужно получать с помощью функции predict_proba)

In [0]:
clf = GradientBoostingClassifier(learning_rate=0.2, n_estimators=36, verbose=True, random_state=241)
clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
res = log_loss(y_test, y_pred)

print(res)
with open("grad_boost.3.txt", "w") as file:
    file.write("%.2f" % res)